In [ ]:
import requests
import time
from datetime import datetime, timedelta

# 🔧 Configuration
API_KEY = ""  # your API key
CAMPAIGN_ID =   # your Yandex Market campaign ID

# 🗓️ Automatically get last month and year
today = datetime.today()
first_day_this_month = today.replace(day=1)
last_month_date = first_day_this_month - timedelta(days=1)
MONTH = str(last_month_date.month)   # previous month as number
YEAR = str(last_month_date.year)     # year of that previous month

print(f"📅 Automatically selected period: {MONTH}/{YEAR}")

BASE_URL = "https://api.partner.market.yandex.ru/v2/reports/goods-realization"
HEADERS = {"Api-Key": API_KEY}

# Step 1: Generate report
payload = {
    "campaignId": CAMPAIGN_ID,
    "month": MONTH,
    "year": YEAR
}

print(f"📤 Generating 'goods-realization' report for {MONTH}/{YEAR}...")
response = requests.post(f"{BASE_URL}/generate", headers=HEADERS, json=payload)

if response.status_code != 200:
    raise Exception(f"❌ Failed to generate report: {response.text}")

report_id = response.json()["result"]["reportId"]
print(f"✅ Report requested. ID: {report_id}")



In [ ]:
import requests
import os
from openpyxl import load_workbook

# 🔧 Configuration
API_KEY = ""  # your API key
REPORT_ID = report_id  # replace with your actual reportId
SAVE_PATH = r""

# Fixed filename
FIXED_FILENAME = ""

# API endpoint
url = f"https://api.partner.market.yandex.ru/v2/reports/info/{REPORT_ID}"
headers = {"Api-Key": API_KEY}

# ------------------------------------------------------------
# 🧾 Fetch report info
# ------------------------------------------------------------
print(f"📄 Fetching info for report ID: {REPORT_ID} ...")
response = requests.get(url, headers=headers)

if response.status_code != 200:
    raise Exception(f"❌ Failed to get report info: {response.text}")

data = response.json()
print("✅ Report Info retrieved.")

# Extract download link
report_info = data.get("result", {})
file_url = report_info.get("file")

if not file_url:
    raise Exception("⚠️ Report file link not found in response.")

# ------------------------------------------------------------
# ⬇️ Download report
# ------------------------------------------------------------
os.makedirs(SAVE_PATH, exist_ok=True)
file_path = os.path.join(SAVE_PATH, FIXED_FILENAME)

print(f"⬇️ Downloading report to: {file_path}")
file_response = requests.get(file_url)

if file_response.status_code == 200:
    with open(file_path, "wb") as f:
        f.write(file_response.content)
    print(f"✅ Report downloaded successfully: {file_path}")
else:
    raise Exception(f"❌ Failed to download report file: {file_response.text}")

# ------------------------------------------------------------
# 🧹 Clean and keep only desired data
# ------------------------------------------------------------
try:
    wb = load_workbook(file_path)
    sheet_to_keep = "Доставленные товары"

    if sheet_to_keep in wb.sheetnames:
        for sheet in wb.sheetnames:
            if sheet != sheet_to_keep:
                del wb[sheet]

        ws = wb[sheet_to_keep]

        # ✅ Step 1: Unmerge all merged cells and fill their values
        merged_ranges = list(ws.merged_cells.ranges)
        for merged_range in merged_ranges:
            top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
            value = top_left_cell.value
            ws.unmerge_cells(str(merged_range))
            for row in ws.iter_rows(min_row=merged_range.min_row, max_row=merged_range.max_row,
                                    min_col=merged_range.min_col, max_col=merged_range.max_col):
                for cell in row:
                    cell.value = value
        print("🔄 Unmerged all merged cells and filled values.")

        # 🧽 Step 2: Delete the first 16 rows
        ws.delete_rows(1, 16)
        print("🧽 Deleted the first 16 rows.")

        # ✅ Step 3: Keep only specific columns
        keep_columns = [
            "Название товара",
            "Доставлено, шт.",
            "Дата оформления заказа",
            "Стоимость всех доставленных штук с НДС без учёта скидок, ₽"
        ]

        header_row = [cell.value for cell in ws[1]]
        keep_indexes = [i + 1 for i, col_name in enumerate(header_row) if col_name in keep_columns]

        if not keep_indexes:
            print("⚠️ None of the specified columns were found in the sheet.")
        else:
            for i in range(ws.max_column, 0, -1):
                if i not in keep_indexes:
                    ws.delete_cols(i)
            print(f"🧹 Kept only selected columns: {keep_columns}")

        # 🚮 Step 4: Remove empty rows (where product name is None)
        rows_to_delete = []
        for row in range(ws.max_row, 1, -1):
            if not ws.cell(row=row, column=1).value:
                rows_to_delete.append(row)
        for row in rows_to_delete:
            ws.delete_rows(row)
        print(f"🧼 Removed {len(rows_to_delete)} empty rows.")

        # 💾 Save workbook
        wb.save(file_path)
        print(f"✅ Final file with 'Sales' column saved: {file_path}")

    else:
        print(f"⚠️ '{sheet_to_keep}' not found in workbook. No sheets removed.")

except Exception as e:
    print(f"⚠️ Failed to modify workbook: {e}")


## Appending Final Output to sharepoint

In [ ]:
import os
import pandas as pd
from openpyxl import load_workbook
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential

# SharePoint link and file details
site_url = ''
doc_library = ''
file_name = ''

# Client ID & Secret
client_id = ""
client_secret = ""

# Define the path to your local Excel file containing the new data
new_data_file_path = r""

# Define the columns to check for duplicates
subset_columns = [
    'Название товара', 'Доставлено, шт.', 'Дата оформления заказа', 'Стоимость всех доставленных штук с НДС без учёта скидок, ₽']

try:
    # Authenticate using Client ID & Secret
    credentials = ClientCredential(client_id, client_secret)
    ctx = ClientContext(site_url).with_credentials(credentials)
    ctx.load(ctx.web)
    ctx.execute_query()
    print("✅ Connected to SharePoint:", ctx.web.properties['Title'])

    # Download the existing file from SharePoint
    response = File.open_binary(ctx, f"{doc_library}/{file_name}")

    # Write the response content to a local file
    with open(file_name, "wb") as existing_file:
        existing_file.write(response.content)

    print("Existing file has been downloaded from SharePoint.")

    # Load the existing data into a DataFrame
    try:
        existing_df = pd.read_excel(file_name)
        print("Existing data loaded into DataFrame.")
    except Exception as e:
        print(f"Error loading existing data: {e}")
        exit(1)

except Exception as e:
    print(f"Error during file download for existing data: {e}")
    exit(1)

# Load the new data into a DataFrame
try:
    df = pd.read_excel(new_data_file_path)
    print("New data loaded into DataFrame.")
except Exception as e:
    print(f"Error loading new data from the file: {e}")
    exit(1)

# Combine the existing data with the new data
try:
    # Combine and deduplicate based on the subset columns
    combined_df = pd.concat([existing_df, df], ignore_index=True)
    combined_df = combined_df.drop_duplicates(subset=subset_columns)
    print("New data merged with existing data successfully.")
except Exception as e:
    print(f"Error while merging data: {e}")
    exit(1)

# Get the new rows added after combining
new_rows = combined_df[len(existing_df):]

if new_rows.empty:
    print("No new rows to add.")
else:
    # Append the new rows to the Excel sheet
    try:
        book = load_workbook(file_name)
        sheet = book['Доставленные товары']  # Pls check that this matches your sheet name
        for row in new_rows.itertuples(index=False, name=None):
            sheet.append(row)

        # Save the modified file
        book.save(file_name)
        print("Data has been successfully written to the Excel file without duplicates.")
    except Exception as e:
        print(f"Error saving the Excel file: {e}")
        exit(1)

    # Upload the updated file back to SharePoint
    try:
        with open(file_name, 'rb') as content_file:
            file_content = content_file.read()

        File.save_binary(ctx, f"{doc_library}/{file_name}", file_content)
        print("File has been uploaded back to SharePoint.")
        print("Everything Completed")
    except Exception as e:
        print(f"Error during file upload: {e}")

# Cleanup: Remove the local file
if os.path.exists(file_name):
    os.remove(file_name)
    print("Local file has been deleted.")